In [21]:
%run ../../data/api_keys.ipynb
%run ../modulos.ipynb
%run ../funciones_temp.ipynb

In [2]:
def request_github(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}"}

    res=requests.get(url,headers=headers)
    return res.json()

In [32]:
def get_pages_repos(min_estrellas, max_estrellas, num_pagina=1):
    return f'https://api.github.com/search/repositories?per_page=100&page={num_pagina}&q=stars%3A{min_estrellas}..{max_estrellas}&sort=forks&order=asc'

In [4]:
headers = {"Authorization": f"Bearer {personal_access_token}","Accept": "application/vnd.github.v3.star+json","content-type": "application/json"}

In [50]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
database=client['promisearch'] # database
collecion_repos=database['repositorios_prueba'] # colección
len(list(collecion_repos.find()))

1

## Inserciones a DB

In [55]:
def obtener_hist(user,repo,atributo,nom_col):

    consulta_url = f"https://api.github.com/repos/{user}/{repo}/{atributo}"
    repos_resp = []

    page=0
    response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page+1)).json() # primeras 100
    repos_resp.extend([re[nom_col] for re in response])

    while len(response)!=0:
        page+=1
        response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page+1)).json()
        repos_resp.extend([re[nom_col] for re in response])

  
    serie_acum=lista_fecha_a_df(repos_resp)
    try:
        cuantiles=pd.qcut(serie_acum, q=4, retbins=True)[1]
        cuant_prop=f'{cuantiles[1]},{",".join([str(round(ai,4)) for ai in list(cuantiles[2:] / cuantiles[1:-1])])}'
    except:
        return '0,0,0,0'

    return cuant_prop

In [34]:
def obtener_info_repo(user,repo):
    """Regresa json con info histórica y del repo"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo

    response1 = request_github_no_json(star_count_url)
     
    cuantiles_estrellas = obtener_hist(user,repo,'stargazers','starred_at')
    cuantiles_forks = obtener_hist(user,repo,'forks','created_at')

    resp_json=response1.json()
    resp_json['cuant_estrellas']=cuantiles_estrellas
    resp_json['cuant_forks']=cuantiles_forks
 
    return resp_json

In [45]:
# res_json=obtener_info_repo('pysentimiento','pysentimiento')
# res_json

# print(res_json.get('id',np.nan))
# print(res_json.get('html_url',np.nan))
# print(res_json.get('cuant_estrellas',np.nan))
# print(res_json.get('cuant_forks',np.nan))
# print(res_json.get('size',np.nan))
# print(res_json.get('stargazers_count',np.nan))
# print(res_json.get('forks',np.nan))
# print(res_json.get('open_issues',np.nan))
# print(res_json.get('subscribers_count',np.nan))
# print(res_json.get('network_count',np.nan))

In [83]:
def obtener_repos_de_pags(min_estrellas,max_estrellas,numero_requests,num_muestreo, k):
    #repos_final = {}
    num_pagina=1 
    
    # Sólo agarrar muestreo de repos
    for _ in range(numero_requests):
        repos_bins = []

        i=0
        while i <1:
            # Construyo la URL
            url_objetivo = get_pages_repos(min_estrellas, max_estrellas, num_pagina)
            #print(num_pagina)
            print(url_objetivo)
            # Aqui hago la peticion a github
            search_github = requests.get(url_objetivo ,headers=headers)
            search_github_json = search_github.json()
            try:
                lista_repos = search_github_json['items'] # 100 repos
                lista_random = random.sample(lista_repos, num_muestreo)   
                print(f'Todo bien {_+1}')             
                repos_bins.extend(lista_random)
                
            except:  
                print('A dormir')
                time.sleep(61)
                i-=1
            i+=1
        num_pagina += 1
        
        #repos_final[f'{_}'] = repos_bins
        collecion_repos.insert_one({f'{k}':repos_bins})
    return None

In [107]:
#avitua = [(100, 200)]
#vizuet = []
#memo = []
collecion_repos.drop()

num_paginas_visitar=1
num_repos_por_pagina=5
num_veces=3
#------Memo----------- 50 
# for k in range(1, 10):
# Avitua
# # for k in range(10, 1000000000):

for k in range(1, num_veces):
    obtener_repos_de_pags(30*k,30*(k+1),num_paginas_visitar,num_repos_por_pagina, k)

https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A0..10&sort=forks&order=asc
Todo bien 1
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A10..20&sort=forks&order=asc
Todo bien 1
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A20..30&sort=forks&order=asc
Todo bien 1


## Creación de csv

In [109]:
id_repos = []
name_repos = []
sizes = []
stargazers_count_list = []
forks_list = []
open_issues_list = []
watchers = []
network_count_list = []
suscribers_list = []
repos1=[]

cuantiles_star=[]
cuantiles_forks=[]
lenguajes=[]

for k in tqdm_notebook(range(num_veces)):
    lista_dict=list(collecion_repos.find({ f'{k}' : { '$exists' : 'true' } }))[0][f'{k}']
    #lista_dict=repos_final[f'{k}']
    for repo in tqdm_notebook(lista_dict, desc='Inner loop'):
        repo_name=repo['name']
        user=repo['owner']['login']
        # --------------------------------------------------
        json_repo=obtener_info_repo(user,repo_name)

        id_repos.append(json_repo.get('id',np.nan))
        name_repos.append(json_repo.get('html_url',np.nan))
        cuantiles_star.append(json_repo.get('cuant_estrellas',np.nan))
        cuantiles_forks.append(json_repo.get('cuant_forks',np.nan))
        lenguajes.append(json_repo.get('language',np.nan))
        sizes.append(json_repo.get('size',np.nan))
        stargazers_count_list.append(json_repo.get('stargazers_count',np.nan))
        forks_list.append(json_repo.get('forks',np.nan))
        open_issues_list.append(json_repo.get('open_issues',np.nan))
        watchers.append(json_repo.get('subscribers_count',np.nan))
        network_count_list.append(json_repo.get('network_count',np.nan)) # número de personas que han "contribuido"

/tmp/ipykernel_95972/3782893636.py:16: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_95972/3782893636.py:19: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Inner loop:   0%|          | 0/3 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/3 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/3 [00:00<?, ?it/s]

In [110]:
repos_df = pd.DataFrame(dict(
    ids=id_repos,
    name_repo=name_repos,
    lenguaje=lenguajes,
    stargazers=stargazers_count_list,
    watchers=watchers,
    cuantiles_star=cuantiles_star,
    cuantiles_forks=cuantiles_forks,
    forks=forks_list,
    # used_by=[],   
    tamaño_kb=sizes,
    issues=open_issues_list,
    network_size=network_count_list
    # pull_request=[],
    # branches=[],
    # no_commits=[]
))

repos_df

,ids,name_repo,lenguaje,stargazers,watchers,cuantiles_star,cuantiles_forks,forks,tamaño_kb,issues,network_size
0,18580501,https://github.com/haridoki/first-repo,None,0,1,"0,0,0,0","0,0,0,0",0,0,0,0
1,18579931,https://github.com/ben1787/DogWalk,Objective-C,0,2,"0,0,0,0","0,0,0,0",0,1352,1,0
2,18579677,https://github.com/erikwaing/ES20_Ghost,TeX,0,3,"0,0,0,0","0,0,0,0",0,372,1,0
3,26798615,https://github.com/tranvictor/lion_attr,Ruby,14,1,"8.0,1.25,1.2,1.1667","0,0,0,0",0,316,1,0
4,26097372,https://github.com/SingleCycleKing/Android-L-W...,Java,10,3,"4.75,1.3684,1.2692,1.2121","0,0,0,0",0,318,0,0
5,19675463,https://github.com/tsdye/harris-matrix,Common Lisp,20,4,"5.5,1.8182,1.45,1.3793","0,0,0,0",0,18675,0,0
6,51178265,https://github.com/damko/xkb_kinesis_advantage...,None,20,2,"5.75,1.8261,1.4524,1.3115","0,0,0,0",0,554,0,0
7,46023027,https://github.com/jtackaberry/flexrouter,None,20,4,"5.75,1.8261,1.4524,1.3115","0,0,0,0",0,109,2,0
8,53880268,https://github.com/muigaiunaka/2013-Portfolio-...,HTML,24,1,"16.0,1.375,1.0227,1.0222","0,0,0,0",0,14331,0,0


In [43]:
repos_df[['star_q1','star_gq2','star_gq3','star_gq4']]=repos_df.cuantiles_star.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_star'],inplace=True)
repos_df[['fork_q1','fork_gq2','fork_gq3','fork_gq4']]=repos_df.cuantiles_forks.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_forks'],inplace=True)
repos_df.tail()

,ids,name_repo,stargazers,watchers,forks,lenguaje,tamaño_kb,issues,network_size,star_q1,star_gq2,star_gq3,star_gq4,fork_q1,fork_gq2,fork_gq3,fork_gq4
45,100889899,https://github.com/johnpatrickmorgan/wtfautola...,1005,11,21,Swift,358,8,21,533.5,1.3046,1.2507,1.1545,6.75,1.7037,1.413,1.2923
46,11455172,https://github.com/elbryan/FFCircularProgressView,1001,43,112,Objective-C,1514,7,112,480.0,1.3958,1.2761,1.1708,34.5,1.7391,1.4083,1.2899
47,122935819,https://github.com/alibaba/G3D,1008,60,110,TypeScript,2532,3,110,529.0,1.3497,1.2185,1.1586,37.75,1.6556,1.396,1.2607
48,4184416,https://github.com/bwsewell/tablecloth,997,80,130,JavaScript,242,10,130,531.75,1.3597,1.2299,1.1212,36.5,1.8356,1.5,1.2935
49,64872804,https://github.com/harshjv/github-repo-size,1011,15,117,JavaScript,1064,30,117,327.25,1.7158,1.4118,1.2753,30.5,1.9016,1.4914,1.3179


In [ ]:
repos_df.to_csv('../../data/data_sin_etiqueta.csv', index=False)